# The Natural Scenes Dataset (NSD)

From [naturalscenesdataset.org](https://naturalscenesdataset.org):
```text
The Natural Scenes Dataset (NSD) is a large-scale fMRI dataset conducted at ultra-high-field (7T) strength at the Center of Magnetic Resonance Research (CMRR) at the University of Minnesota. The dataset consists of whole-brain, high-resolution (1.8-mm isotropic, 1.6-s sampling rate) fMRI measurements of 8 healthy adult subjects while they viewed thousands of color natural scenes over the course of 30–40 scan sessions. While viewing these images, subjects were engaged in a continuous recognition task in which they reported whether they had seen each given image at any point in the experiment. These data constitute a massive benchmark dataset for computational models of visual representation and cognition, and can support a wide range of scientific inquiry.
```

The NSD was spear-headed by Kendrick Kay at the University of Minnesota, and is one of the best datasets for examining in-depth visual function due to the large quantity of scan time for each participant.

The NSD is hosted on Amazon S3, so we will access it using the `S3Path` and `S3Client` interface of `cloudpathlib`. The NSD does not require authentication for access, so we do not need to load or name our credentials.

In [ ]:
from cloudpathlib import S3Path, S3Client
from pathlib import Path

# Set up our cache path:
cache_path = Path('/tmp/cache')
if not cache_path.exists():
    cache_path.mkdir()

# Create the root S3Path for the NSD:
nsd_base_path = S3Path(
    's3://natural-scenes-dataset/',
    client=S3Client(
        no_sign_request=True,
        local_cache_dir=cache_path))

Let's look around inside of the NSD S3 bucket...

In [ ]:
from utilities import ls, crawl

ls(nsd_base_path)

In [ ]:
ls(nsd_base_path / 'nsddata' / 'freesurfer')

In [ ]:
ls(nsd_base_path / 'nsddata' / 'ppdata' / 'subj01' / 'func1pt8mm')

## Loading an NSD file: The Coefficient of Determination (or Variance Explained)

We can load a nifti file from the NSD using the cloudpath object, and cloudpath's ability to convert a cloudpath into a local filepath (by using `cloud_path_object.fspath`). The NSD is not in BIDS format, but its data are organized relatively intuitively. The directory `ppdata` in particular holeds the preprocessed data for the subjects. Here we will extract the coefficient of determination (also called the r-squared or the percentage of variance explained) of the population receptive field (PRF) models for subject 1.

In [ ]:
import nibabel as nib

nsd_pppath = nsd_base_path / 'nsddata' / 'ppdata'
cod_filename = nsd_pppath / 'subj01' / 'func1pt8mm' / 'prf_R2.nii.gz'
cod = nib.load(cod_filename.fspath)
cod.shape

In [ ]:
# Once we have loaded the CoD as a nifti object, we can plot it using matplotlib.

import matplotlib.pyplot as plt

plt.imshow(cod.dataobj[:,:,40], vmin=0, vmax=100, cmap='hot')

### Loading Subjects using neuropythy

Because the NSD is on S3, neuropythy can also access it and load FreeSurfer subjects from it. To do this, we can simply direct neuropythy to the S3 FreeSurfer subject directory (note that this will not share cache with the `cloudpathlib` library).

In [ ]:
# Import neuropythy
import neuropythy as ny

# Tell neuropythy where we want to keep cache data.
ny.config['data_cache_root'] = '/tmp/cache'

In [ ]:
# Where is the FreeSurfer data for a subject?

ls(nsd_base_path / 'nsddata' / 'freesurfer' / 'subj01')

In [ ]:
# Ask neuropythy to load a FreeSurfer subject:
sub = ny.freesurfer_subject(
    's3://natural-scenes-dataset/nsddata/freesurfer/subj01')

In [ ]:
# Have neuropythy plot this subject:
ny.cortex_plot((sub.lh, sub.rh), surface='white')